# Validation of calibration results

## import libraries


In [1]:
import os
import sys
import datetime
import time
import glob
import string

import numpy as np
import pandas as pd
import xarray as xr

from configparser import ConfigParser
import hydroStats


import matplotlib.pyplot as plt
import plotly
from plotly import graph_objs as go, offline as po, tools
from plotly.subplots import make_subplots
import plotly.express as px

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## reading settings file

In [22]:
iniFile = "P:/watmodel/CWATM/calibration/multi_UB2/settings_all_28.txt"

(drive, path) = os.path.splitdrive(iniFile)
(path, fil) = os.path.split(path)
print (">> Reading settings file (" + fil + ")...")

parser = ConfigParser()
parser.read(iniFile)

root = parser.get('DEFAULT', 'Root')

ForcingStart = datetime.datetime.strptime(parser.get('DEFAULT', 'ForcingStart'), "%d/%m/%Y")  # Start of forcing
ForcingEnd = datetime.datetime.strptime(parser.get('DEFAULT', 'ForcingEnd'), "%d/%m/%Y")  # Start of forcing

WarmupDays = int(parser.get('DEFAULT', 'WarmupDays'))
CatchmentDataPath = os.path.join(root, parser.get('Path', 'CatchmentDataPath'))
SubCatchmentPath = os.path.join(root, parser.get('Path', 'SubCatchmentPath'))

path_temp = os.path.join(root, parser.get('Path', 'Temp'))
path_maps = os.path.join(root, os.path.join(parser.get('Path', 'CatchmentDataPath'), "maps_pcraster"))
path_result = os.path.join(root, parser.get('Path', 'Result'))

Qtss_csv = os.path.join(root, parser.get('CSV', 'Qtss'))
Qgis_csv = os.path.join(root, parser.get('CSV', 'Qgis'))
Qgis_out = os.path.join(root, parser.get('CSV', 'QgisOut'))



>> Reading settings file (settings_all_28.txt)...


## Select Station

In [23]:
from ipywidgets import interact, widgets
from IPython.display import display

w = widgets.IntSlider(value=1,min=1,max=28,step=1,description='Station No:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
display(w)


IntSlider(value=1, continuous_update=False, description='Station No:', max=28, min=1)

### Calculate

In [131]:
station_no = int(w.value)
loc = station_no - 1

print (">> Reading Qgis2.csv file...")
stationdata = pd.read_csv(os.path.join(Qgis_csv), sep=",", index_col=0)
stationdata = stationdata.iloc[loc]

Cal_Start = datetime.datetime.strptime(stationdata['Cal_Start'], "%d/%m/%Y")
Cal_End = datetime.datetime.strptime(stationdata['Cal_End'], "%d/%m/%Y")
Val_Start = datetime.datetime.strptime(stationdata['Val_Start'], "%d/%m/%Y")
Val_End = datetime.datetime.strptime(stationdata['Val_End'], "%d/%m/%Y")

Val2_Start = datetime.datetime.strptime("01/06/2014", "%d/%m/%Y")
Val2_End = datetime.datetime.strptime("31/07/2018", "%d/%m/%Y")

#Cal_End = datetime.datetime.strptime("31/10/2009", "%d/%m/%Y")

#stationdata
# Load observed streamflow
streamflow_data = pd.read_csv(Qtss_csv, sep=",", index_col=0, parse_dates=True, dayfirst=True)

Qobs = streamflow_data[stationdata['ID']]
Qobs[Qobs < 0] = np.NaN


dates_cal  = Qobs[Cal_Start:Cal_End].index
dates_val  = Qobs[Val_Start:Val_End].index
dates_val2  = Qobs[Val2_Start:Val2_End].index
Qobs_cal = Qobs[Cal_Start:Cal_End]
Qobs_val = Qobs[Val_Start:Val_End]
Qobs_val2 = Qobs[Val2_Start:Val2_End]

name = stationdata['ID'] + " " + stationdata['Station']
#stationdata
max1 = np.nanmax([Qobs_cal])
max2 = np.nanmax([Qobs_val])
max = np.nanmax([max1,max2])



>> Reading Qgis2.csv file...


In [132]:

#path_subcatch = os.path.join(SubCatchmentPath, stationdata['ID'])
path_subcatch = os.path.join(SubCatchmentPath, "G0001")

    
Qsim1 = pd.read_csv(os.path.join(path_subcatch, "streamflow_simulated_best.tss"), sep=",", parse_dates=True, index_col=0, header=None)

# monthly
Qsim = Qsim1.resample('MS').mean()
Qsim = Qsim.loc[:, 1]
Qsim_cal = Qsim[Cal_Start:Cal_End]
Qsim_val = Qsim[Val_Start:Val_End]


# ----------------
d = pd.date_range('1995-6-1','2007-10-31', freq='MS')

ts1_cal =0
"""

filename ="P:/watmodel/CWATM/calibration/multi_UB/runall/out2/discharge_monthavg.tss"
Qsim = np.loadtxt(filename, skiprows=19)
s1 = Qsim[:,station_no]
ts1 = pd.Series(s1, index=d)
ts1_cal = ts1[Cal_Start:Cal_End]
ts1_val = ts1[Val_Start:Val_End]

filename ="P:/watmodel/CWATM/calibration/multi_UB/runall/out2/discharge_monthavg.tss"
Qsim = np.loadtxt(filename, skiprows=19)
s2 = Qsim[:,station_no]
ts2 = pd.Series(s2, index=d)
ts2_cal = ts2[Cal_Start:Cal_End]
ts2_val = ts2[Val_Start:Val_End]

"""

'\n\nfilename ="P:/watmodel/CWATM/calibration/multi_UB/runall/out2/discharge_monthavg.tss"\nQsim = np.loadtxt(filename, skiprows=19)\ns1 = Qsim[:,station_no]\nts1 = pd.Series(s1, index=d)\nts1_cal = ts1[Cal_Start:Cal_End]\nts1_val = ts1[Val_Start:Val_End]\n\nfilename ="P:/watmodel/CWATM/calibration/multi_UB/runall/out2/discharge_monthavg.tss"\nQsim = np.loadtxt(filename, skiprows=19)\ns2 = Qsim[:,station_no]\nts2 = pd.Series(s2, index=d)\nts2_cal = ts2[Cal_Start:Cal_End]\nts2_val = ts2[Val_Start:Val_End]\n\n'

## Statistical values
### Observation vs. simulation

In [133]:
def geo_idx(dd, dd_array):
   """
     search for nearest decimal degree in an array of decimal degrees and return the index.
     np.argmin returns the indices of minium value along an axis.
     so subtract dd from all values in dd_array, take absolute value and find index of minium.
   """
   geo_idx = (np.abs(dd_array - dd)).argmin()
   return geo_idx

filename = "P:/watmodel/CWATM/calibration/multi_UB2/runall/out/discharge_monthavg.nc"
filename = "C:/Users/smilovic/Downloads/discharge_monthavg (1).nc"
#file = xr.open_dataarray(name,decode_times=False)
file = xr.open_dataset(filename,decode_times=False)
reference_date = '1995-06-01'
d = pd.date_range(start=reference_date, periods=file.sizes['time'], freq='MS')

lats = file['lat'][:]
lons = file['lon'][:]

in_lat = stationdata['lat_corr']
in_lon = stationdata['lon_corr']

lat_idx = geo_idx(in_lat, lats)
lon_idx = geo_idx(in_lon, lons)

v= file["discharge_monthavg"][:,lat_idx,lon_idx]
#val= file['discharge_monthavg']
#v = file["discharge_monthavg"][0:10,1,1]
ts1 = pd.Series(v, index=d)
ts1_cal = ts1[Cal_Start:Cal_End]
ts1_val = ts1[Val_Start:Val_End]
ts1_val2 = ts1[Val2_Start:Val2_End]
file.close()

#filename = "P:/watmodel/CWATM/calibration/multi_UB2/runall/discharge_monthavg.nc"
filename = "C:/Users/smilovic/Downloads/discharge_monthavg (6).nc"
#file = xr.open_dataarray(name,decode_times=False)
file = xr.open_dataset(filename,decode_times=False)
reference_date = '1995-06-01'
d = pd.date_range(start=reference_date, periods=file.sizes['time'], freq='MS')

lats = file['lat'][:]
lons = file['lon'][:]

in_lat = stationdata['lat_corr']
in_lon = stationdata['lon_corr']

lat_idx = geo_idx(in_lat, lats)
lon_idx = geo_idx(in_lon, lons)

v= file["discharge_monthavg"][:,lat_idx,lon_idx]
#val= file['discharge_monthavg']
#v = file["discharge_monthavg"][0:10,1,1]
ts2 = pd.Series(v, index=d)
ts2_cal = ts2[Cal_Start:Cal_End]
ts2_val = ts2[Val_Start:Val_End]
ts2_val2 = ts2[Val2_Start:Val2_End]
file.close()


## Figure

In [134]:

max1 = np.nanmax(ts2_cal)
max2 = np.nanmax(ts2_val)
max = np.nanmax([max,max1,max2])


fig = go.Figure()

fig.add_trace(go.Scatter(y=Qobs_cal,x=dates_cal,mode='lines+markers', name='Observed',  marker = dict(color='blue', size=8), line = dict(color='blue', width=2)))
#fig.add_trace(go.Scatter(y=Qsim_cal,x=dates_cal,mode='lines', name='Simulated',  line = dict(color='red', width=4, dash='dot')))
#fig.add_trace(go.Scatter(y=ts1_cal,x=dates_cal,mode='lines', name='SimMod30', line = dict(color='orange', width=2)))
fig.add_trace(go.Scatter(y=ts2_cal,x=dates_cal,mode='lines', name='Simulated', line = dict(color='green', width=2)))
#fig.add_trace(go.Scatter(y=ts2_cal,x=dates_cal,mode='lines', name='SimMod7', line = dict(color='red', width=2)))
#fig.add_trace(go.Scatter(y=ts3_cal,x=dates_cal,mode='lines', name='Netcdf_7'))

fig.update_yaxes(range=[0, max])
fig.update_layout(title='Calibration at '+ name, xaxis_title = 'Month', yaxis_title = 'Discharge (m3/s) mean')
#fig.update_xaxes(rangeslider_visible=True)
fig.show()
#------

fig = go.Figure()

fig.add_trace(go.Scatter(y=Qobs_val,x=dates_val,mode='lines+markers', name='Observed',  marker = dict(color='blue', size=8), line = dict(color='blue', width=2)))
#fig.add_trace(go.Scatter(y=Qsim_val,x=dates_val,mode='lines', name='Simulated',  line = dict(color='red', width=4, dash='dot')))
#fig.add_trace(go.Scatter(y=ts1_val,x=dates_val,mode='lines', name='SimMod30', line = dict(color='orange', width=2)))
fig.add_trace(go.Scatter(y=ts2_val,x=dates_val,mode='lines', name='Simulated', line = dict(color='green', width=2)))
#fig.add_trace(go.Scatter(y=ts2_val,x=dates_val,mode='lines', name='SimMod7', line = dict(color='red', width=2)))
#fig.add_trace(go.Scatter(y=ts3_val,x=dates_val,mode='lines', name='Netcdf_7'))

fig.update_yaxes(range=[0, max])
fig.update_layout(title='Validation at '+ name, xaxis_title = 'Month', yaxis_title = 'Discharge (m3/s) mean')

fig.show()


#fig = go.Figure()

#fig.add_trace(go.Scatter(y=Qobs_val2,x=dates_val2,mode='lines+markers', name='Observed',  marker = dict(color='blue', size=8), line = dict(color='blue', width=2)))
##fig.add_trace(go.Scatter(y=Qsim_val,x=dates_val,mode='lines', name='Simulated',  line = dict(color='red', width=4, dash='dot')))
##fig.add_trace(go.Scatter(y=ts1_val,x=dates_val,mode='lines', name='SimMod30', line = dict(color='orange', width=2)))
##fig.add_trace(go.Scatter(y=ts2_val,x=dates_val,mode='lines', name='SimMod7', line = dict(color='green', width=2)))
#fig.add_trace(go.Scatter(y=ts2_val2,x=dates_val2,mode='lines', name='SimMod7', line = dict(color='red', width=2)))
##fig.add_trace(go.Scatter(y=ts3_val,x=dates_val,mode='lines', name='Netcdf_7'))

#fig.update_yaxes(range=[0, max])
#fig.update_layout(title='Validation at '+ name, xaxis_title = 'Month', yaxis_title = 'Discharge (m3/s)')

#fig.show()


## Statistical values
### Observation vs. simulation

In [135]:
import pandas as pd


o = Qobs_cal.values
s = []
s2= []
for i in range(len(o)):
    if not (np.isnan(o[i])):
        #s.append(Qsim_cal[i])
        s.append(ts2_cal[i])
        s2.append(ts2_cal[i])
scal = np.asarray(s)
scal2 = np.asarray(s2)
ocal = o[~np.isnan(o)]

o = Qobs_val.values
s = []
s2 = []
for i in range(len(o)):
    if not (np.isnan(o[i])):
        #s.append(Qsim_val[i])
        s.append(ts2_val[i])
        s2.append(ts2_val[i])
sval = np.asarray(s)
sval2 = np.asarray(s)
oval = o[~np.isnan(o)]


print (stationdata['ID'],"-", stationdata['Station'])
df=pd.DataFrame({"Obs Cal.":['',
                        '',
                        '',
                        '',
                        '',
                        '',
                        '',
                         "{0:.0f}".format(np.mean(ocal)),
                        "{0:.0f}".format(np.min(ocal)),
                        "{0:.0f}".format(np.percentile(ocal,5)),
                        "{0:.0f}".format(np.percentile(ocal,50)),
                        "{0:.0f}".format(np.percentile(ocal,95)),
                        "{0:.0f}".format(np.percentile(ocal,99)),
                        "{0:.0f}".format(np.max(ocal))                         
                       ],
                 "Sim Cal.":["{0:.2f}".format(hydroStats.KGE(s=scal, o=ocal, warmup=0)),
                        "{0:.2f}".format(hydroStats.NS(s=scal, o=ocal, warmup=0)),
                        "{0:.2f}".format(hydroStats.correlation(s=scal, o=ocal, warmup=0)),
                        "{0:.1f}".format(hydroStats.pc_bias2(s=scal, o=ocal, warmup=0)),
                        "{0:.0f}".format(hydroStats.rmse(s=scal, o=ocal, warmup=0)),
                        "{0:.0f}".format(hydroStats.mae(s=scal, o=ocal, warmup=0)),
                        "",
                        "{0:.0f}".format(np.mean(scal)),
                        "{0:.0f}".format(np.min(scal)),
                        "{0:.0f}".format(np.percentile(scal,5)),
                        "{0:.0f}".format(np.percentile(scal,50)),
                        "{0:.0f}".format(np.percentile(scal,95)),
                        "{0:.0f}".format(np.percentile(scal,99)),
                        "{0:.0f}".format(np.max(scal))                        
                       ],
                "Obs Val.":['',
                        '',
                        '',
                        '',
                        '',
                        '',
                        '',
                         "{0:.0f}".format(np.mean(oval)),
                        "{0:.0f}".format(np.min(oval)),
                        "{0:.0f}".format(np.percentile(oval,5)),
                        "{0:.0f}".format(np.percentile(oval,50)),
                        "{0:.0f}".format(np.percentile(oval,95)),
                        "{0:.0f}".format(np.percentile(oval,99)),
                        "{0:.0f}".format(np.max(oval))                         
                       ],
                 "Sim Val.":["{0:.2f}".format(hydroStats.KGE(s=sval, o=oval, warmup=0)),
                        "{0:.2f}".format(hydroStats.NS(s=sval, o=oval, warmup=0)),
                        "{0:.2f}".format(hydroStats.correlation(s=sval, o=oval, warmup=0)),
                        "{0:.1f}".format(hydroStats.pc_bias2(s=sval, o=oval, warmup=0)),
                        "{0:.0f}".format(hydroStats.rmse(s=sval, o=oval, warmup=0)),
                        "{0:.0f}".format(hydroStats.mae(s=sval, o=oval, warmup=0)),
                        "",
                        "{0:.0f}".format(np.mean(sval)),
                        "{0:.0f}".format(np.min(sval)),
                        "{0:.0f}".format(np.percentile(sval,5)),
                        "{0:.0f}".format(np.percentile(sval,50)),
                        "{0:.0f}".format(np.percentile(sval,95)),
                        "{0:.0f}".format(np.percentile(sval,99)),
                        "{0:.0f}".format(np.max(sval))                        
                       ]})

df.columns =['Obs cal','Sim cal','Obs val','Sim val']
df.index = ['KGE','NS',r'R$^{2}$','Bias','RMSE','MAE',' ','Mean',"Min","5%","50%","95%","99%","Max"]

df.to_clipboard(excel=True)
df


G04 - Nighoje


,Obs cal,Sim cal,Obs val,Sim val
KGE,,0.64,,0.23
NS,,0.80,,-0.11
R$^{2}$,,0.92,,0.62
Bias,,18.9,,57.0
RMSE,,30,,49
MAE,,23,,36
,,,,
Mean,67,80,47,74
Min,1,7,2,11
5%,4,15,4,24


In [93]:
o = Qobs_cal.values
s = []
for i in range(len(o)):
    if not (np.isnan(o[i])):
        s.append(Qsim_cal[i])
        #s.append(ts2_cal[i])
s1 = np.asarray(s)
o1 = o[~np.isnan(o)]
KGE = "{0:.2f}".format(hydroStats.KGE(s=s1, o=o1, warmup=WarmupDays))
bias = "{0:.2f}".format(hydroStats.pc_bias2(s=s1, o=o1, warmup=WarmupDays))
corr = "{0:.2f}".format(hydroStats.correlation(s=s1, o=o1, warmup=WarmupDays))
NS = "{0:.2f}".format(hydroStats.NS(s=s1, o=o1, warmup=WarmupDays))
NSlog = "{0:.2f}".format(hydroStats.NSlog(s=s1, o=o1, warmup=WarmupDays))
#print ("Objective function for original calibration") 
#print ("KGE,bias,corr,NS,NSlog") 
#print(KGE,bias,corr,NS,NSlog)

mean_o =  "{0:.2f}".format(np.mean(o1))
mean_s =  "{0:.2f}".format(np.mean(s1))

# ------------------
o = Qobs_val.values
s = []
for i in range(len(o)):
    if not (np.isnan(o[i])):
        s.append(Qsim_val[i])
        #s.append(ts2_val[i])
s1 = np.asarray(s)
o1 = o[~np.isnan(o)]
KGE = "{0:.2f}".format(hydroStats.KGE(s=s1, o=o1, warmup=WarmupDays))
bias = "{0:.2f}".format(hydroStats.pc_bias2(s=s1, o=o1, warmup=WarmupDays))
corr = "{0:.2f}".format(hydroStats.correlation(s=s1, o=o1, warmup=WarmupDays))
NS = "{0:.2f}".format(hydroStats.NS(s=s1, o=o1, warmup=WarmupDays))
NSlog = "{0:.2f}".format(hydroStats.NSlog(s=s1, o=o1, warmup=WarmupDays))

#print(KGE,bias,corr,NS,NSlog)



In [94]:
o = Qobs_cal.values
s = []
s2=[]
for i in range(len(o)):
    if not (np.isnan(o[i])):
        #s.append(Qsim_cal[i])
        s.append(ts1_cal[i])
        #s2.append(ts2_cal[i])
scal = np.asarray(s)
#scal2 = np.asarray(s2)
ocal = o[~np.isnan(o)]
KGE = "{0:.2f}".format(hydroStats.KGE(s=scal, o=ocal, warmup=0))
bias = "{0:.2f}".format(hydroStats.pc_bias2(s=scal, o=ocal, warmup=0))
corr = "{0:.2f}".format(hydroStats.correlation(s=scal, o=ocal, warmup=0))
NS = "{0:.2f}".format(hydroStats.NS(s=scal, o=ocal, warmup=0))
NSlog = "{0:.2f}".format(hydroStats.NSlog(s=scal, o=ocal, warmup=0))

print ("Objective function for 30 days GW model") 
print ("KGE,bias,corr,NS,NSlog") 
print("Cal: ",KGE,bias,corr,NS,NSlog)

mean_o =  "{0:.2f}".format(np.mean(ocal))
mean_s =  "{0:.2f}".format(np.mean(scal))
#print(mean_o,mean_s)

# ------------------
o = Qobs_val.values
s = []
s2 = []
for i in range(len(o)):
    if not (np.isnan(o[i])):
        #s.append(Qsim_val[i])
        s.append(ts1_val[i])
        #s2.append(ts2_val[i])
sval = np.asarray(s)
#sval2 = np.asarray(s)
oval = o[~np.isnan(o)]
KGE = "{0:.2f}".format(hydroStats.KGE(s=sval, o=oval, warmup=0))
bias = "{0:.2f}".format(hydroStats.pc_bias2(s=sval, o=oval, warmup=0))
corr = "{0:.2f}".format(hydroStats.correlation(s=sval, o=oval, warmup=0))
NS = "{0:.2f}".format(hydroStats.NS(s=sval, o=oval, warmup=0))
NSlog = "{0:.2f}".format(hydroStats.NSlog(s=sval, o=oval, warmup=0))

print("Val: ",KGE,bias,corr,NS,NSlog)

Objective function for 30 days GW model
KGE,bias,corr,NS,NSlog
Cal:  0.05 90.23 0.89 -0.20 -0.33
Val:  -0.24 113.42 0.62 -1.44 -0.61


In [95]:
o = Qobs_cal.values
s = []
s2=[]
for i in range(len(o)):
    if not (np.isnan(o[i])):
        #s.append(Qsim_cal[i])
        s.append(ts2_cal[i])
        s2.append(ts2_cal[i])
scal = np.asarray(s)
scal2 = np.asarray(s2)
ocal = o[~np.isnan(o)]
KGE = "{0:.2f}".format(hydroStats.KGE(s=scal, o=ocal, warmup=0))
bias = "{0:.2f}".format(hydroStats.pc_bias2(s=scal, o=ocal, warmup=0))
corr = "{0:.2f}".format(hydroStats.correlation(s=scal, o=ocal, warmup=0))
NS = "{0:.2f}".format(hydroStats.NS(s=scal, o=ocal, warmup=0))
NSlog = "{0:.2f}".format(hydroStats.NSlog(s=scal, o=ocal, warmup=0))

print ("Objective function for 7 days GW model") 
print ("KGE,bias,corr,NS,NSlog") 
print("Cal: ",KGE,bias,corr,NS,NSlog)

mean_o =  "{0:.2f}".format(np.mean(ocal))
mean_s =  "{0:.2f}".format(np.mean(scal))
#print(mean_o,mean_s)

# ------------------
o = Qobs_val.values
s = []
s2 = []
for i in range(len(o)):
    if not (np.isnan(o[i])):
        #s.append(Qsim_val[i])
        s.append(ts2_val[i])
        s2.append(ts2_val[i])
sval = np.asarray(s)
sval2 = np.asarray(s)
oval = o[~np.isnan(o)]
KGE = "{0:.2f}".format(hydroStats.KGE(s=sval, o=oval, warmup=0))
bias = "{0:.2f}".format(hydroStats.pc_bias2(s=sval, o=oval, warmup=0))
corr = "{0:.2f}".format(hydroStats.correlation(s=sval, o=oval, warmup=0))
NS = "{0:.2f}".format(hydroStats.NS(s=sval, o=oval, warmup=0))
NSlog = "{0:.2f}".format(hydroStats.NSlog(s=sval, o=oval, warmup=0))

print("Val: ",KGE,bias,corr,NS,NSlog)



Objective function for 7 days GW model
KGE,bias,corr,NS,NSlog
Cal:  -0.07 101.93 0.90 -0.44 -0.38
Val:  -0.41 130.62 0.65 -1.75 -0.82
